In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from datetime import datetime

In [ ]:
#DEAR -> REPORTS -> SALES BY PRODUCT DETAILS -> [select time frame] -> EXPORT -> EXCEL WITHOUT TOTALS

df = pd.read_excel('2021details.xlsx',engine='openpyxl',header=0,skiprows=5, parse_dates=False)#parse_dates didn't work, see below
print(df.iloc[30:40])#head is full of junk right now lol
#print(df.head())

In [ ]:
#start cleaning some of the unused or needed columns / rows, converting "Invoice Date" to datetime objects
df = df.drop(['SKU', 'COGS','Profit'], axis=1, inplace=False)#not trusted data columns or useful, we'll ignore them for now
list_of_retail_stores = ['Customer Square ' + str(x) for x in range(1,4)] #we have 3 stores active, we won't count the Warehouse (CS4?)
df = df[df.Customer.isin(list_of_retail_stores) & df.Product.notnull()] #remove wholesale and weird null anomoly entries
df = df.rename(columns={'Sale':'Price','Invoice Date':'Date'})#renaming for legibility 
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')#get these dates converted from string to something more powerful

print(df.head(8))

In [ ]:
#quick wine filter setup and check to make sure there's no items with bad categories that need fixing

active_categories = df.Category.unique().tolist()
active_categories.sort()
print(active_categories)
wine_list = ['White','Orange Wine','Bubbles','Red New World','Red Old World','Rose']#active categories just for vino
whiskey_list = ['Whiskey American Single Malt', 'Whiskey Bourbon/American', 'Whiskey Canadian', 'Whiskey French', 'Whiskey Indian', 'Whiskey Irish', 'Whiskey Japanese', 'Whiskey Mexican', 'Whiskey Rye', 'Whiskey Scotch', 'Whiskey Taiwaneese', 'Whisky German']
#if there's bad categories (at item creation in DEAR) fix in DEAR and re-run report - easiest for now or manually edit here
print(df.groupby(['Product','Customer',(pd.Grouper(key='Date', freq='1M'))]).Quantity.sum().head()) #example

In [ ]:
#lets make a plot showing the change in category sales over the year
linegraph = df.groupby(['Category',(pd.Grouper(key='Date', freq='1M'))]).Quantity.sum().reset_index()#example
print(linegraph)

fig, ax = plt.subplots()
for cat in wine_list:
    ax.plot(linegraph[linegraph.Category == cat].Date,linegraph[linegraph.Category == cat].Quantity,label = cat)

ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax.xaxis.set_minor_locator(mdates.MonthLocator())

plt.xlabel('Date')
plt.ylabel('Number Sold')
plt.title('Category Sales Over Time')
plt.legend()#show it!
plt.show()
